# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice


In [ ]:
import os
if "training" not in os.listdir():
    os.mkdir("./training")

In [ ]:
%%writefile train.py
from sklearn.linear_model import LogisticRegression
import argparse
import os
import numpy as np
from sklearn.metrics import mean_squared_error
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from azureml.core.run import Run
from azureml.core import Dataset, Workspace

# download data from source
dataset = pd.read_csv("https://raw.githubusercontent.com/cpaulisi/nd00333-capstone/master/starter_file/wine.csv")

def preprocess(data): 
    y_df = data.pop("quality").apply(lambda x: 1 if x == "good" else 0)
    return data, y_df
    
x, y = preprocess(dataset)
# TODO: Split data into train and test sets.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=11)
run = Run.get_context()

def main():
  # Add arguments to script
    parser = argparse.ArgumentParser()
    parser.add_argument('--C', type=float, default=1.0, help="Inverse of regularization strength. Smaller values cause stronger regularization")
    parser.add_argument('--max_iter', type=int, default=100, help="Maximum number of iterations to converge")
    args = parser.parse_args()
    run.log("Regularization Strength:", np.float(args.C))
    run.log("Max iterations:", np.int(args.max_iter))
    model = LogisticRegression(C=args.C, max_iter=args.max_iter).fit(x_train, y_train)
    accuracy = model.score(x_test, y_test)
    run.log("Accuracy", np.float(accuracy))
    os.makedirs('output', exist_ok=True) 
    joblib.dump(model, 'output/hdr_model.joblib')
if __name__ == '__main__':
    main()

In [ ]:
!cp train.py ./training

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
ws.write_config(path='.azureml')
exp = Experiment(workspace=ws, name="hdr-exp")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
# assuming cluster name does not already exist, provision for cluster
cluster_name = "cluster-compute"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Using existing cluster...")
except ComputeTargetException:
    print("Creating cluster " + cluster_name+ "...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_V2', 
        vm_priority='lowpriority', 
        max_nodes=3
    )
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
early_termination_policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1, delay_evaluation=2)
param_sampling =  RandomParameterSampling(
    {
        "--C": uniform(0.005, 1),
        "--max_iter": choice(100, 200, 500, 1000)
    }
)


# Create a SKLearn estimator for use with train.py
estimator = SKLearn(
    source_directory="./training",
    compute_target = cpu_cluster,
    pip_packages=['azureml-sdk','argparse', ''],
    conda_packages=['pandas','ipykernel', 'numpy'],
    entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=6,
    max_concurrent_runs=3

)

In [ ]:
#TODO: Submit your experiment
hdr = exp.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hdr).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run_hdr = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run_hdr.get_metrics()
parameter_values = best_run_hdr.get_details()['runDefinition']['arguments']
print('run_id: ', best_run_hdr.id)

print(' Accuracy:', best_run_metrics['Accuracy'])
print(' Parameter_Values:',parameter_values)


In [ ]:
#TODO: Save the best model
os.makedirs('output', exist_ok=True)
joblib.dump(value=parameter_values, filename='output/hdr_model_parameters.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
hyderdrive_model = best_run_hdr.register_model(model_name='hyperdrive_model')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

